In [ ]:
#| hide
from l402.payment_providers import *
from l402.payment_clients import *
from l402.utils import *

# l402-python

> 

## Usage

### Installation

Install latest from [pypi](https://pypi.org/project/l402/)

```sh
$ pip install l402
```

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/Fewsats/l402-python
[docs]: https://Fewsats.github.io/l402-python/
[pypi]: https://pypi.org/project/l402-python/
[conda]: https://anaconda.org/Fewsats/l402-python

## How to use

In [ ]:
#| hide
from fastapi.responses import JSONResponse
from http import HTTPStatus
from cdp import *
from fastapi import FastAPI
from fastcore.all import first, dict2obj

import httpx


In [ ]:
# | hide
# Create FastAPI app
app = FastAPI()

server = ServerManager(app, port=9000).start()

INFO:     Started server process [23855]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9000 (Press CTRL+C to quit)


In [ ]:
#| hide 
from uuid import uuid4
import json
import os

In [ ]:
#| hide

# w =create_test_wallet(fund=False, chain='base-mainnet')
# import json
# json.dump(w.export_data().to_dict(), open('index-seed.json', 'w'))
# !cat index-seed.json
Cdp.configure(api_key_name=os.getenv("CDP_KEY_NAME"), private_key=os.getenv("CDP_PRIVATE_KEY"))

w = create_test_wallet(fund=False, chain='base-sepolia')
# w = Wallet.import_data(WalletData.from_dict(json.load(open('index-seed.json'))))
w.balance('usdc')

Decimal('0')

In [ ]:
#| hide
ps = PaymentServer(
    payment_request_url="http://localhost:9000/payment_request",
    onchain_provider=w,
)

@app.get("/offers")
def offers():
    offers_list = [Offer(
        amount=1,
        currency='USD',
        description='Purchase 1 credit for API access',
        offer_id=str(uuid4()),
        payment_methods=['onchain'], # maybe not needed because we can generate it from the payment provider
        title='1 Credit Package',
        type='one-time'
    )]

    
    offers_response = ps.create_offers(offers_list)
    return JSONResponse(
        content=offers_response.model_dump(),
        status_code=402
    )

@app.post("/payment_request")
async def create_payment_request(request: PaymentRequest):
    try:
        payment_request = ps.create_payment_request(**request.model_dump())
        return JSONResponse(
            content=payment_request,
            status_code=200
        )
    except Exception as e:
        print(e)

### Paying with the L402

The L402 supports multiple payment providers. In this example we'll 
use a coinbase wallet as    an onchain provider. This wallet will be used 
to pay for L402 offers.

In [ ]:
from l402.payment_clients import Client, CoinbaseProvider
from cdp import Wallet

In [ ]:
# CDP can be directly used as an onchain provider.
w = Wallet.create()

c = Client(onchain_provider=CoinbaseProvider(wallet=w))

Next we retrieve an offer from a demo server. This will return a 402 status code and a JSON response with the available offers.

In [ ]:
r1 = httpx.get("http://localhost:9000/offers")
r1.status_code, r1.json()

(402,
 {'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': '58e1ee0a-e283-4f80-8a41-5c2f0ea138ca',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': '42f46106-8989-4ed1-a770-adf9ad64c30b',
  'payment_request_url': 'http://localhost:9000/payment_request',
  'version': '0.2.2'})

In [ ]:
#| hide
if r1.status_code == HTTPStatus.PAYMENT_REQUIRED:
    try:
        ps = c.pay(r1.json())
    except Exception as e:
        print(e)

Insufficient funds: have 0, need 1.


To pay, we just need to provide the L402 response to the client and it will automatically use configured payment provider that matches the payment method.

```python
if r1.status_code == HTTPStatus.PAYMENT_REQUIRED:
    ps = c.pay(r1.json())
```

### Getting paid with the L402

The L402 protocol implements a two-step payment flow:

1. The server returns a `402 Payment Required` status code and information about the available offers.
2. The client uses the L402 response to fetch the information for a specific payment method.

In this example, we implement this using a server with two endpoints:

1. `/offers` - Returns available payment options with 402 status
2. `/payment_request` - Generates payment details for chosen payment method

```python
from l402.payment_providers import PaymentServer, Offer

app = FastAPI()


ps = PaymentServer(
    payment_request_url="http://localhost:9000/payment_request",
    onchain_provider=w,
)

@app.get("/offers")
def offers():
    offers_list = [Offer(
        amount=1,
        currency='USD',
        description='Purchase 1 credit for API access',
        offer_id=str(uuid4()),
        payment_methods=['onchain'],
        title='1 Credit Package',
        type='one-time'
    )]
    
    offers_response = ps.create_offers(offers_list)
    # Return the status code 402 Payment Required with the offers
    return JSONResponse(content=offers_response.model_dump(), status_code=402)

@app.post("/payment_request")
async def create_payment_request(request: PaymentRequest):
    payment_request = ps.create_payment_request(**request.model_dump())
    return JSONResponse(content=payment_request)
```



In [ ]:
r = httpx.get("http://localhost:9000/offers")
offers_data = r.json()
r.status_code, offers_data

(402,
 {'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': '1808e12c-1824-49d8-a348-278959f6008d',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': '82dbf89a-b18f-4737-b350-c58731b1cc18',
  'payment_request_url': 'http://localhost:9000/payment_request',
  'version': '0.2.2'})

The client can use this offer to request the information for a payment method. We have to choose the payment method and, in the case of coinbase, the chain and asset.

In [ ]:
offers_data = dict2obj(offers_data)
data = {
    "offer_id": offers_data.offers[0].offer_id, 
    "payment_method": 'onchain',
    "chain": 'base-sepolia', # choose the chain where you want to pay
    "asset": 'usdc', # choose the asset you want to pay
    "payment_context_token": offers_data.payment_context_token 
}
r = httpx.post(offers_data.payment_request_url, json=data)
r, r.json()

(<Response [200 OK]>,
 {'expires_at': '2025-01-30T21:03:46.502392+00:00',
  'offer_id': '1808e12c-1824-49d8-a348-278959f6008d',
  'payment_request': {'address': '0x6EC717D99534aF3917E25De1024f9e5BcDc1B262',
   'chain': 'base-sepolia',
   'asset': 'usdc'},
  'version': '0.2.2'})